In [1]:
from pathlib import Path
from skimage.io import imread, imsave
from cellpose import models, io, utils
import os
import numpy as np
from tnia.deeplearning.dl_helper import quantile_normalization 
import matplotlib.pyplot as plt

In [2]:
parent_path = r'D:\images\tnia-python-images\imagesc\2024_10_11_tough_cellpose_2'
overlay_path = os.path.join(parent_path, 'overlay_2') 
model_path = os.path.join(parent_path, 'models')

model = models.CellposeModel(gpu=True, model_type=None, pretrained_model= os.path.join(model_path, 'cellpose_for_protrusions_3'))


In [3]:
print('diameter mean', model.diam_labels, model.diam_mean)

diameter mean 60.48541 30.0


In [4]:
def mask_overlay(img, masks, colors=None):
    """Overlay masks on image (set image to grayscale).

    Args:
        img (int or float, 2D or 3D array): Image of size [Ly x Lx (x nchan)].
        masks (int, 2D array): Masks where 0=NO masks; 1,2,...=mask labels.
        colors (int, 2D array, optional): Size [nmasks x 3], each entry is a color in 0-255 range.

    Returns:
        RGB (uint8, 3D array): Array of masks overlaid on grayscale image.
    """
    if colors is not None:
        if colors.max() > 1:
            colors = np.float32(colors)
            colors /= 255
        colors = utils.rgb_to_hsv(colors)
    if img.ndim > 2:
        img = img.astype(np.float32).mean(axis=-1)
    else:
        img = img.astype(np.float32)

    HSV = np.zeros((img.shape[0], img.shape[1], 3), np.float32)
    HSV[:, :, 2] = np.clip((img / 255. if img.max() > 1 else img) * 1.5, 0, 1)
    hues = np.linspace(0, 1, masks.max() + 1)[np.random.permutation(masks.max())]
    for n in range(int(masks.max())):
        ipix = (masks == n + 1).nonzero()
        if colors is None:
            HSV[ipix[0], ipix[1], 0] = hues[n]
        else:
            HSV[ipix[0], ipix[1], 0] = colors[n, 0]
        HSV[ipix[0], ipix[1], 1] = 1.0
    RGB = (utils.hsv_to_rgb(HSV) * 255).astype(np.uint8)
    return RGB

In [5]:
if not Path(overlay_path).exists():
    Path(overlay_path).mkdir()

files = Path(parent_path).glob('*.tif')

for file in files:
    print(file)
    img = imread(file)
    img = quantile_normalization(img).astype(np.float32)

    masks = model.eval(img, channels=[2, 3], cellprob_threshold=-1, flow_threshold = 0.5, normalize = False)

    img_ = (255)*(img/img.max())
    img_ = img_.astype(np.uint8)

    outlines = utils.masks_to_outlines(masks[0])
    overlay = mask_overlay(img_, masks[0])
    imsave(overlay_path + '\\' + file.name, overlay)
    


D:\images\tnia-python-images\imagesc\2024_10_11_tough_cellpose_2\img101.tif
D:\images\tnia-python-images\imagesc\2024_10_11_tough_cellpose_2\img102.tif
D:\images\tnia-python-images\imagesc\2024_10_11_tough_cellpose_2\img103.tif
D:\images\tnia-python-images\imagesc\2024_10_11_tough_cellpose_2\img104.tif
D:\images\tnia-python-images\imagesc\2024_10_11_tough_cellpose_2\img105.tif
D:\images\tnia-python-images\imagesc\2024_10_11_tough_cellpose_2\img110.tif
D:\images\tnia-python-images\imagesc\2024_10_11_tough_cellpose_2\img111.tif
D:\images\tnia-python-images\imagesc\2024_10_11_tough_cellpose_2\img112.tif
D:\images\tnia-python-images\imagesc\2024_10_11_tough_cellpose_2\img113.tif
D:\images\tnia-python-images\imagesc\2024_10_11_tough_cellpose_2\img125.tif
D:\images\tnia-python-images\imagesc\2024_10_11_tough_cellpose_2\img126.tif
D:\images\tnia-python-images\imagesc\2024_10_11_tough_cellpose_2\img128.tif
D:\images\tnia-python-images\imagesc\2024_10_11_tough_cellpose_2\img129.tif
D:\images\tn